Dataset: https://github.com/arnauddri/hn

In [1]:
import pandas as pd
submissions = pd.read_csv("stories.csv")
submissions.columns = ["submission_time", "upvotes", "url", "headline"]
submissions = submissions.dropna()
submissions.head(3)

,submission_time,upvotes,url,headline
0,2015-02-20T11:34:22.000Z,1,startupjuncture.com,24sessions: live business advice over video-chat
1,2015-02-20T11:35:32.000Z,3,blog.erratasec.com,Some notes on SuperFish
2,2015-02-20T11:36:18.000Z,1,twitter.com,Apple Watch models could contain 29.16g of gold


In [2]:
tokenized_headlines = []
for item in submissions["headline"]:
    tokenized_headlines.append(item.split(" "))

In [ ]:
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []
for item in tokenized_headlines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)

In [ ]:
import numpy as np
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)

counts = pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)

In [ ]:
# clean_tokenized and counts have been loaded in.
for i, item in enumerate(clean_tokenized):
    for token in item:
        if token in unique_tokens:
            counts.iloc[i][token] += 1

In [ ]:
# clean_tokenized and counts have been loaded in.
word_counts = counts.sum(axis=0)

counts = counts.loc[:,(word_counts >= 5) & (word_counts <= 100)]

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, submissions["upvotes"], test_size=0.2, random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

In [ ]:

mse = sum((y_test - predictions) ** 2) / len(predictions)